In [ ]:
import torch
import torch.nn as nn
#from .utils import load_state_dict_from_url
from torch.hub import load_state_dict_from_url
model_urls = {
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
}

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
	return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=dilation, groups=groups, bias=False, dilation=dilation)

def conv1x1(in_planes, out_planes,stride=1):
	return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class Bottleneck(nn.Module):
	expansion = 4

	def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1, base_width=64, dilation=1, norm_layer=None):
		super(Bottleneck, self).__init__()
		if norm_layer is None:
			norm_layer = nn.BatchNorm2d
		width = int(planes * (base_width / 64.)) * groups

		self.conv1 = conv1x1(inplanes, width)
		self.bn1 = norm_layer(width)
		self.conv2 = conv3x3(width, width, stride=stride, groups=groups, dilation=dilation)
		self.bn2 = norm_layer(width)
		self.conv3 = conv1x1(width, planes*self.expansion)
		self.bn3 = norm_layer(planes*self.expansion)
		self.relu = nn.ReLU(inplace=True)
		self.downsample = downsample
		self.stride = stride

	def forward(self, x):
		identify = x

		out = self.conv1(x)
		out = self.bn1(out)
		out = self.relu(out)

		out = self.conv2(out)
		out = self.bn2(out)
		out = self.relu(out)

		out = self.conv3(out)
		out = self.bn3(out)
		
		if self.downsample is not None:
			identify = self.downsample(x)

		out += identify
		out = self.relu(out)

		return out

class ResNet(nn.Module):
	def __init__(self, block, layers, num_classes=1000, zero_init_residual=False, groups=1, width_per_group=64, replace_stride_with_dilation=None, norm_layer=None):
		super(ResNet, self).__init__()
		if norm_layer is None:
			norm_layer = nn.BatchNorm2d
		self._norm_layer = norm_layer

		self.inplanes = 64
		self.dilation = 1
		if replace_stride_with_dilation is None:
			replace_stride_with_dilation = [False, False, False]
		if len(replace_stride_with_dilation) != 3:
			raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
		self.groups = groups
		self.base_width = width_per_group

		self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
		self.bn1 = norm_layer(self.inplanes)
		self.relu = nn.ReLU(inplace=True)
		self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

		self.layer1 = self._make_layer(block, 64, layers[0])
		self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
		self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
		self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])

		self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
		self.fc = nn.Linear(512*block.expansion, num_classes)

		for m in self.modules():
			if isinstance(m, nn.Conv2d):
				nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
			elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
				nn.init.constant_(m.weight, 1)
				nn.init.constant_(m.bias, 0)
		if zero_init_residual:
			for m in self.modules():
				if isinstance(m, Bottleneck):
					nn.init.constant_(m.bn3.weight, 0)
				elif isinstance(m, BasicBlock):
					nn.init.constant_(m.bn2.weight, 0)

	def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
		norm_layer = self._norm_layer
		downsample = None
		previous_dilation = self.dilation
		if dilate:
			self.dilation *= stride
			stride = 1
		if stride != 1 or self.inplanes != planes*block.expansion:
			downsample = nn.Sequential(
				conv1x1(self.inplanes, planes*block.expansion, stride),
				norm_layer(planes*block.expansion),
				)

		layers = []
		layers.append(block(self.inplanes, planes, stride=stride, downsample=downsample, groups=self.groups, base_width=self.base_width, dilation=previous_dilation, norm_layer=norm_layer))
		self.inplanes = planes * block.expansion
		for _ in range(1, blocks):
			layers.append(block(self.inplanes, planes, groups=self.groups, base_width=self.base_width, dilation=self.dilation, norm_layer=norm_layer))

		return nn.Sequential(*layers)

	def _forward_impl(self, x):
		x = self.conv1(x)
		x = self.bn1(x)
		x = self.relu(x)
		x = self.maxpool(x)

		x = self.layer1(x)
		x = self.layer2(x)
		x = self.layer3(x)
		x = self.layer4(x)

		x = self.avgpool(x)
		x = torch.flatten(x, 1)
		x = self.fc(x)

		return x

	def forward(self, x):
		return self._forward_impl(x)

def _resnet(arch, block, layers, pretrained, progress, **kwargs):
	model = ResNet(block, layers, **kwargs)
	if pretrained:
		state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
		model.load_state_dict(state_dict)
	return model

def resnet101(pretrained=False, progress=True, **kwargs):
	return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader 


BATCH_SIZE = 64
EPOCH = 20
LR = 0.001

my_tf = transforms.Compose([
	transforms.Resize((224,224)),  #模型的输入图像尺寸
	transforms.ToTensor(),         
	transforms.Normalize([0.4914, 0.4822, 0.4465],[0.247, 0.243, 0.261]) 
	])

train_dataset = torchvision.datasets.CIFAR10(root='./', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./', train=False, transform=transforms.ToTensor(), download=True)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset)

model = resnet101(pretrained=True)
model.fc = nn.Linear(2048, 10)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(EPOCH):
	for idx, data in enumerate(train_dataloader):
		inputs, labels = data
		inputs = inputs.to(device)
		labels = labels.to(device)
		output = model(inputs)
		loss = criterion(output, labels)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
	print('epoch{} loss:{:.4f}'.format(epoch+1, loss.item()))

#保存模型参数
torch.save(model, 'cifar10_resnet.pt')
#模型加载
model = torch.load('cifar10_resnet.pt')
#测试
model.eval()
correct, total = 0, 0
for data in test_dataloader:
	inputs, labels = data
	inputs = inputs.to(device)
	labels = labels.to(device)
	output = model(inputs)
	_,idx = torch.max(output.data,1) # 输出最大值的位置
	total += labels.size(0) # 全部图片
	correct +=(idx==labels).sum() # 正确的图片
print(f"accuracy:{100.*correct/total}")

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth



epoch1 loss:1.3510
epoch2 loss:1.0140
epoch3 loss:0.4693
epoch4 loss:0.9694
epoch5 loss:1.3922
epoch6 loss:0.8600
epoch7 loss:0.6453
epoch8 loss:0.8224
epoch9 loss:0.9556
epoch10 loss:0.5807
epoch11 loss:0.1239
epoch12 loss:0.2607
epoch13 loss:1.1403
epoch14 loss:0.2009
epoch15 loss:0.3298
epoch16 loss:0.1103
epoch17 loss:0.4401
epoch18 loss:0.5281
epoch19 loss:0.3840
epoch20 loss:0.2466


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bottleneck. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


accuracy:74.79999542236328
